### 23-10-24 Update 

> quick update. I've been chatting with Lucas Ferreira and he pointed out that Jay's group uploaded BAM files from 10X Genomics, which can be correctly converted back to FASTQ files. I'm going to explore this option, since that would allow us to finally properly reprocess the reads. Stay tuned

This is the `prefetch` command that downloads BAM files that can be converted to FASTQ files

```prefetch --type TenX --max-size 100000000 <SRR_Accession>```

`prefetch` is available in the `alex_py3_dev` conda environment.

It should replace the `aws` related commands below

#### Relevant SRAs for the Low MOI experiments:

GSM3417251 - Pilot Screen (low MOI) transcriptomes scRNA-seq:

`SRR7967475
SRR7967476
SRR7967477
SRR7967478
SRR7967479
SRR7967480`

GSM3417252 - Pilot Screen (low MOI) gRNA enrichment scRNA-seq:

`SRR7967481`

GSM3417253 - Pilot Screen (high MOI) transcriptomes scRNA-seq:

`SRR7967482
SRR7967483
SRR7967484
SRR7967485
SRR7967486
SRR7967487`

GSM3417254 - Pilot Screen (high MOI) gRNA enrichment scRNA-seq:

`SRR7967488
SRR7967489
SRR7967490
SRR7967491
SRR7967492
SRR7967493`

#### Download BAM files using `prefetch`

In [1]:
%%writefile /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/list_of_SRR_low_high_MOI.txt
SRR7967475
SRR7967476
SRR7967477
SRR7967478
SRR7967479
SRR7967480
SRR7967481
SRR7967482
SRR7967483
SRR7967484
SRR7967485
SRR7967486
SRR7967487
SRR7967488
SRR7967489
SRR7967490
SRR7967491
SRR7967492
SRR7967493

Writing /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/list_of_SRR_low_high_MOI.txt


In [18]:
%%bash
cd /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq
source /data/reddylab/software/miniconda3/bin/activate alex_py3_dev
sbatch -o /data/gersbachlab/Revathy/collabs/Susan/logs/download.low_high_MOI_screen_data.%a.out \
    --mem=8G \
    --array=0-19 \
    <<'EOF'
#!/bin/bash
srrs=($(cat /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/list_of_SRR_low_high_MOI.txt))
srr=${srrs[${SLURM_ARRAY_TASK_ID}]}
prefetch $srr --type TenX --max-size 1000000000 
EOF

Submitted batch job 30300680


In [33]:
%%bash
mkdir /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/test
cd /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/test
source /data/reddylab/software/miniconda3/bin/activate alex_py3_dev
sbatch -o /data/gersbachlab/Revathy/collabs/Susan/logs/download.test.out \
    --mem=8G \
    <<'EOF'
#!/bin/bash
srr=SRR7967481
prefetch $srr --type TenX --max-size 1000000000 
EOF

Submitted batch job 30443984


In [40]:
!ls /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/test/SRR7967481

pilot_lowmoi_screen.TATTGCTG.grna.bam


Transform BAM to FASTQ files

In [9]:
%%bash
cd /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi
sbatch -o /data/gersbachlab/Revathy/collabs/Susan/logs/download.low_high_MOI_screen_data.%a.out \
    --mem=8G \
    --array=0-6 \
    <<'EOF'
#!/bin/bash
SRRs=(SRR7967475 SRR7967476 SRR7967477 SRR7967478 SRR7967479 SRR7967480 SRR7967481)
SRR=${SRRs[${SLURM_ARRAY_TASK_ID}]}
BAM=($(/bin/ls -1 /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi/${SRR}/*.bam))
/data/reddylab/software/cellranger/cellranger-7.1.0/lib/bin/bamtofastq \
    /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi/${SRR}/*.bam \
    ${BAM/.bam/} 
EOF

Submitted batch job 30302717


In [11]:
%%bash
cd /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi
sbatch -o /data/gersbachlab/Revathy/collabs/Susan/logs/download.low_high_MOI_screen_data.bam_to_fastq.%a.out \
    --mem=8G \
    --array=0-11 \
    <<'EOF'
#!/bin/bash
SRRs=(SRR7967482 SRR7967483 SRR7967484 SRR7967485 SRR7967486 SRR7967487 SRR7967488 SRR7967489 SRR7967490 SRR7967491 SRR7967492 SRR7967493)
SRR=${SRRs[${SLURM_ARRAY_TASK_ID}]}
BAM=($(/bin/ls -1 /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi/${SRR}/*.bam))
/data/reddylab/software/cellranger/cellranger-7.1.0/lib/bin/bamtofastq \
    /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi/${SRR}/*.bam \
    ${BAM/.bam/} 
EOF

Submitted batch job 30302725


In [41]:
%%bash
cd /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/test/SRR7967481
sbatch -o /data/gersbachlab/Revathy/collabs/Susan/logs/download.bamtofastq.SRR7967481.out \
    --mem=8G \
    <<'EOF'
#!/bin/bash
BAM=($(/bin/ls -1 /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/test/SRR7967481/*.bam))
/data/reddylab/software/cellranger/cellranger-7.1.0/lib/bin/bamtofastq \
    /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/test/SRR7967481/*.bam \
    ${BAM/.bam/} 
EOF

Submitted batch job 30443987


### Create metadata spreadsheets -  library.csv and feature_ref.csv for low_moi data 

Run this through cellranger using the CRISPR modality

* [ ] Have to check if data needs to be demultiplexed

In [1]:
!mkdir /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata

In [1]:
%%writefile /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/low_moi_lib.csv
fastqs,sample,library_type
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r1_SI-GA-F1_HGMFLBGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r1_SI-GA-F1_HGTKKBGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r1_SI-GA-F1_HGTV3BGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r1_SI-GA-F1_HMWTJBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r2_SI-GA-F2_HGMFLBGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r2_SI-GA-F2_HGTKKBGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r2_SI-GA-F2_HGTV3BGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r2_SI-GA-F2_HMWTJBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r3_SI-GA-F3_HGMFLBGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r3_SI-GA-F3_HGTKKBGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r3_SI-GA-F3_HGTV3BGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r3_SI-GA-F3_HMWTJBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r4_SI-GA-F4_HGMFLBGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r4_SI-GA-F4_HGTKKBGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r4_SI-GA-F4_HGTV3BGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r4_SI-GA-F4_HMWTJBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r5_SI-GA-F5_HGMFLBGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r5_SI-GA-F5_HGTKKBGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r5_SI-GA-F5_HGTV3BGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r5_SI-GA-F5_HMWTJBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r6_SI-GA-F6_HGMFLBGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r6_SI-GA-F6_HGTKKBGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r6_SI-GA-F6_HGTV3BGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r6_SI-GA-F6_HMWTJBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_gRNA_TATTGCTG_HGMFLBGX2bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_gRNA_TATTGCTG_HGTKKBGX2bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_gRNA_TATTGCTG_HGTV3BGX2bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_gRNA_TATTGCTG_HMWTJBGX3bamtofastq,CRISPR Guide Capture

Writing /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/low_moi_lib.test_1.csv


In [3]:
!rm -r /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/high_moi/

In [5]:
%%bash
mkdir -p /data/gersbachlab/Revathy/collabs/Susan/results
mkdir -p /data/gersbachlab/Revathy/collabs/Susan/results/gasperini
mkdir -p /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/low_moi

In [17]:
%%bash
cd /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/low_moi
samples=($(cut -f2 -d, '/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/low_moi_lib.csv' | tail -n+2))
sbatch \
    --array=0-28 \
    -p all \
    -o /data/gersbachlab/Revathy/collabs/Susan/logs/low_moi_data_cellranger.%a.out \
    --mem=128G \
    --cpus-per-task=24 \
    <<'EOF'
#!/bin/bash
samples=($(cut -f2 -d, '/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/low_moi_lib.csv' | tail -n+2))
sample=${samples[${SLURM_ARRAY_TASK_ID}]}
/data/gersbachlab/Ruhi/pipeline_perturbseq_like/cellranger-7.1.0/bin/cellranger count \
     --id=${sample} \
     --libraries=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/low_moi_lib.csv \
     --transcriptome=/data/reddylab/Reference_Data/cellranger/refdata-gex-GRCh38-2020-A \
     --feature-ref=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/feature_reference.csv \
     --chemistry=auto \
     --localcores=24 \
     --localmem=64 
EOF

Submitted batch job 30374116


### Test on one sample of Low_moi

In [32]:
!head /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/feature_reference.csv

id,name,read,pattern,sequence,feature_type,target_gene_name
scrambled_1_g1,scrambled_1_g1,R2,(BC)NNNNNNNNNNNNNNNNNNNN,AGCCTAACGATCGGACCGAG,CRISPR Guide Capture,scrambled
scrambled_2_g2,scrambled_2_g2,R2,(BC)NNNNNNNNNNNNNNNNNNNN,TGTCCAGTTCCGTAGGATGG,CRISPR Guide Capture,scrambled
scrambled_3_g3,scrambled_3_g3,R2,(BC)NNNNNNNNNNNNNNNNNNNN,TATTCGTACCGGGCAGCAGG,CRISPR Guide Capture,scrambled
scrambled_4_g4,scrambled_4_g4,R2,(BC)NNNNNNNNNNNNNNNNNNNN,CGGTGGGGCCGCTCACTAGG,CRISPR Guide Capture,scrambled
scrambled_5_g5,scrambled_5_g5,R2,(BC)NNNNNNNNNNNNNNNNNNNN,CTGAACTTCGGTAAAGGCGG,CRISPR Guide Capture,scrambled
scrambled_6_g6,scrambled_6_g6,R2,(BC)NNNNNNNNNNNNNNNNNNNN,CTCACCGTAGAGACGGCACG,CRISPR Guide Capture,scrambled
scrambled_7_g7,scrambled_7_g7,R2,(BC)NNNNNNNNNNNNNNNNNNNN,ACCGCTGCCCTAACCACTGG,CRISPR Guide Capture,scrambled
scrambled_8_g8,scrambled_8_g8,R2,(BC)NNNNNNNNNNNNNNNNNNNN,GCTGTATATCGGCGCCCCGG,CRISPR Guide Capture,scrambled
scrambled_9_g9,scrambled_9_g9,R2,(BC)NNNNNNNNNNNNNNNNNNNN,AT

In [8]:
%%bash
gzip -dc /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi/K1000_CRISPRi_cells_r1_SI-GA-F1_HGMFLBGX2bamtofastq_S1_L001_R1_001.fastq.gz \
| 

@NS500488:481:HGMFLBGX2:1:11310:19587:7308 1:N:0:0
GACCTGGGTTGAACTCGGACTTGGAC
+
A6AAAEEEEEEEEEEEEEEEEEEEEE
@NS500488:481:HGMFLBGX2:1:22211:6898:2848 1:N:0:0
CTAATGGGTGGGTCAACAGGTTTCCA
+
AA6AAAEEEEEEEEEEAEEEEEEEEE
@NS500488:481:HGMFLBGX2:1:23312:6465:9095 1:N:0:0
GGAATAAGTAGAGTGCATGCTTCCGT


In [11]:
%%bash
gzip -dc /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi/K1000_CRISPRi_gRNA_TATTGCTG_HGMFLBGX2bamtofastq_S1_L001_R2_001.fastq.gz \
| grep AGCCTAACGATCGGACCGAG | wc -l

5363


In [18]:
!ls /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi/K1000_CRISPRi_cells_r1*L001_R2*fastq.gz

/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi/K1000_CRISPRi_cells_r1_SI-GA-F1_HGMFLBGX2bamtofastq_S1_L001_R2_001.fastq.gz
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi/K1000_CRISPRi_cells_r1_SI-GA-F1_HGTKKBGX2bamtofastq_S1_L001_R2_001.fastq.gz
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi/K1000_CRISPRi_cells_r1_SI-GA-F1_HGTV3BGX2bamtofastq_S1_L001_R2_001.fastq.gz
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi/K1000_CRISPRi_cells_r1_SI-GA-F1_HMWTJBGX3bamtofastq_S1_L001_R2_001.fastq.gz


In [21]:
%%bash
files=( \
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi/K1000_CRISPRi_gRNA_TATTGCTG_HGMFLBGX2bamtofastq_S1_L001_R2_001.fastq.gz \
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi/K1000_CRISPRi_gRNA_TATTGCTG_HGTKKBGX2bamtofastq_S1_L001_R2_001.fastq.gz \
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi/K1000_CRISPRi_gRNA_TATTGCTG_HGTV3BGX2bamtofastq_S1_L001_R2_001.fastq.gz \
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi/K1000_CRISPRi_gRNA_TATTGCTG_HMWTJBGX3bamtofastq_S1_L001_R2_001.fastq.gz)
for file in ${files[@]};
do
gzip -dc $file | grep GCTGTATATCGGCGCCCCGG | wc -l
done

8353
8888
5389
6460


In [3]:
%%writefile /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/low_moi_lib.test_4.csv
fastqs,sample,library_type
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r1_SI-GA-F1_HGMFLBGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r1_SI-GA-F1_HGTKKBGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r1_SI-GA-F1_HGTV3BGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r1_SI-GA-F1_HMWTJBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_gRNA_TATTGCTG_HGMFLBGX2bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_gRNA_TATTGCTG_HGTKKBGX2bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_gRNA_TATTGCTG_HGTV3BGX2bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_gRNA_TATTGCTG_HMWTJBGX3bamtofastq,CRISPR Guide Capture

Overwriting /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/low_moi_lib.test_4.csv


In [4]:
%%writefile /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/low_moi_lib.test_5.csv
fastqs,sample,library_type
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r2_SI-GA-F2_HGMFLBGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r2_SI-GA-F2_HGTKKBGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r2_SI-GA-F2_HGTV3BGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r2_SI-GA-F2_HMWTJBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_gRNA_TATTGCTG_HGMFLBGX2bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_gRNA_TATTGCTG_HGTKKBGX2bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_gRNA_TATTGCTG_HGTV3BGX2bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_gRNA_TATTGCTG_HMWTJBGX3bamtofastq,CRISPR Guide Capture

Writing /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/low_moi_lib.test_5.csv


In [59]:
%%writefile /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/low_moi_lib.test_3.csv
fastqs,sample,library_type
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r1_SI-GA-F1_HGMFLBGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r2_SI-GA-F2_HGMFLBGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r3_SI-GA-F3_HGMFLBGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r4_SI-GA-F4_HGMFLBGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r5_SI-GA-F5_HGMFLBGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_cells_r6_SI-GA-F6_HGMFLBGX2bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_gRNA_TATTGCTG_HGMFLBGX2bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_gRNA_TATTGCTG_HGTKKBGX2bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_gRNA_TATTGCTG_HGTV3BGX2bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi,K1000_CRISPRi_gRNA_TATTGCTG_HMWTJBGX3bamtofastq,CRISPR Guide Capture

Writing /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/low_moi_lib.test_3.csv


In [60]:
%%bash
cd /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/low_moi
sbatch \
    -p all \
    -o /data/gersbachlab/Revathy/collabs/Susan/logs/low_moi_data_cellranger_test.out \
    --mem=128G \
    --cpus-per-task=24 \
    <<'EOF'
#!/bin/bash
/data/gersbachlab/Ruhi/pipeline_perturbseq_like/cellranger-7.1.0/bin/cellranger count \
     --id="HGMFLBGX2" \
     --libraries=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/low_moi_lib.test_3.csv \
     --transcriptome=/data/reddylab/Reference_Data/cellranger/refdata-gex-GRCh38-2020-A \
     --feature-ref=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/feature_reference_bc_14n.csv \
     --chemistry=auto \
     --localcores=24 \
     --localmem=64 
EOF

Submitted batch job 30443997


In [1]:
!ls /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/

feature_reference_23N_BC.csv  feature_reference.csv	 low_moi_lib.test_2.csv
feature_reference_bc_13n.csv  feature_reference_old.csv  low_moi_lib.test_3.csv
feature_reference_bc_14n.csv  high_moi_lib.csv		 low_moi_lib.test_4.csv
feature_reference_bc_20n.csv  low_moi_lib.csv		 low_moi_lib.test_5.csv
feature_reference_bc_23n.csv  low_moi_lib.test_1.csv


In [9]:
%%bash
cd /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/low_moi
sbatch \
    -p all \
    -o /data/gersbachlab/Revathy/collabs/Susan/logs/low_moi_data_cellranger_test5.out \
    --mem=128G \
    --cpus-per-task=24 \
    <<'EOF'
#!/bin/bash
/data/gersbachlab/Ruhi/pipeline_perturbseq_like/cellranger-7.1.0/bin/cellranger count \
     --id="run1" \
     --libraries=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/low_moi_lib.test_4.csv \
     --transcriptome=/data/reddylab/Reference_Data/cellranger/refdata-gex-GRCh38-2020-A \
     --feature-ref=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/feature_reference_bc_14n.csv \
     --chemistry=auto \
     --localcores=24 \
     --localmem=64 
EOF

Submitted batch job 30445068


In [8]:
!rm -r /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/low_moi/run1

In [25]:
%%bash
cd /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/low_moi
sbatch \
    -p all \
    -o /data/gersbachlab/Revathy/collabs/Susan/logs/low_moi_data_cellranger_test2.out \
    --mem=128G \
    --cpus-per-task=24 \
    <<'EOF'
#!/bin/bash
/data/gersbachlab/Ruhi/pipeline_perturbseq_like/cellranger-7.1.0/bin/cellranger count \
     --id="r1_SI-GA-F1_HGMFLBGX2_HGTKKBGX2_test" \
     --libraries=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/low_moi_lib.test_2.csv \
     --transcriptome=/data/reddylab/Reference_Data/cellranger/refdata-gex-GRCh38-2020-A \
     --feature-ref=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/feature_reference_old.csv \
     --chemistry=auto \
     --localcores=24 \
     --localmem=64 
EOF

Submitted batch job 30443982


In [4]:
!head /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/feature_reference_bc_23n.csv

id,name,read,pattern,sequence,feature_type,target_gene_name
scrambled_1_g1,scrambled_1_g1,R2,(BC)NNNNNNNNNNNNNNNNNNNNNNN,AGCCTAACGATCGGACCGAG,CRISPR Guide Capture,scrambled
scrambled_2_g2,scrambled_2_g2,R2,(BC)NNNNNNNNNNNNNNNNNNNNNNN,TGTCCAGTTCCGTAGGATGG,CRISPR Guide Capture,scrambled
scrambled_3_g3,scrambled_3_g3,R2,(BC)NNNNNNNNNNNNNNNNNNNNNNN,TATTCGTACCGGGCAGCAGG,CRISPR Guide Capture,scrambled
scrambled_4_g4,scrambled_4_g4,R2,(BC)NNNNNNNNNNNNNNNNNNNNNNN,CGGTGGGGCCGCTCACTAGG,CRISPR Guide Capture,scrambled
scrambled_5_g5,scrambled_5_g5,R2,(BC)NNNNNNNNNNNNNNNNNNNNNNN,CTGAACTTCGGTAAAGGCGG,CRISPR Guide Capture,scrambled
scrambled_6_g6,scrambled_6_g6,R2,(BC)NNNNNNNNNNNNNNNNNNNNNNN,CTCACCGTAGAGACGGCACG,CRISPR Guide Capture,scrambled
scrambled_7_g7,scrambled_7_g7,R2,(BC)NNNNNNNNNNNNNNNNNNNNNNN,ACCGCTGCCCTAACCACTGG,CRISPR Guide Capture,scrambled
scrambled_8_g8,scrambled_8_g8,R2,(BC)NNNNNNNNNNNNNNNNNNNNNNN,GCTGTATATCGGCGCCCCGG,CRISPR Guide Capture,scrambled
scrambled_9_g9,scrambled_9_g9,R2,(BC

In [6]:
!head /data/reddylab/Alex/collab/20181203_Tyler/data/metadata/gasperini.low_moi.feature_reference.csv

id,name,read,pattern,sequence,feature_type,target_gene_name
scrambled_1_g1,scrambled_1_g1,R2,NNNNNNNNNNNNNNNAACACCG(BC),AGCCTAACGATCGGACCGAG,CRISPR Guide Capture,scrambled
scrambled_2_g2,scrambled_2_g2,R2,NNNNNNNNNNNNNNNAACACCG(BC),TGTCCAGTTCCGTAGGATGG,CRISPR Guide Capture,scrambled
scrambled_3_g3,scrambled_3_g3,R2,NNNNNNNNNNNNNNNAACACCG(BC),TATTCGTACCGGGCAGCAGG,CRISPR Guide Capture,scrambled
scrambled_4_g4,scrambled_4_g4,R2,NNNNNNNNNNNNNNNAACACCG(BC),CGGTGGGGCCGCTCACTAGG,CRISPR Guide Capture,scrambled
scrambled_5_g5,scrambled_5_g5,R2,NNNNNNNNNNNNNNNAACACCG(BC),CTGAACTTCGGTAAAGGCGG,CRISPR Guide Capture,scrambled
scrambled_6_g6,scrambled_6_g6,R2,NNNNNNNNNNNNNNNAACACCG(BC),CTCACCGTAGAGACGGCACG,CRISPR Guide Capture,scrambled
scrambled_7_g7,scrambled_7_g7,R2,NNNNNNNNNNNNNNNAACACCG(BC),ACCGCTGCCCTAACCACTGG,CRISPR Guide Capture,scrambled
scrambled_8_g8,scrambled_8_g8,R2,NNNNNNNNNNNNNNNAACACCG(BC),GCTGTATATCGGCGCCCCGG,CRISPR Guide Capture,scrambled
scrambled_9_g9,scrambled_9_g9,R2,NNNNNNNNNNN

In [3]:
len('NNNNNNNNNNNNNNNAACACCG')

22

## High_MOI

Create high_moi_lib.csv, should be the same feature reference file 

In [1]:
%%bash
cat /data/reddylab/Alex/collab/20181203_Tyler/data/metadata/gasperini.low_moi.feature_reference.csv \
> /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi.feature_reference.csv

In [7]:
%%writefile /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi_lib.csv
fastqs,sample,library_type
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r1_SI-GA-G1_HMMKLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r1_SI-GA-G1_HMMMLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r1_SI-GA-G1_HW5MKBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r1_SI-GA-G1_HWVT7BGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r2_SI-GA-G2_HMMKLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r2_SI-GA-G2_HMMMLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r2_SI-GA-G2_HW5MKBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r2_SI-GA-G2_HWVT7BGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r3_SI-GA-G3_HMMKLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r3_SI-GA-G3_HMMMLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r3_SI-GA-G3_HW5MKBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r3_SI-GA-G3_HWVT7BGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r4_SI-GA-G4_HMMKLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r4_SI-GA-G4_HMMMLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r4_SI-GA-G4_HW5MKBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r4_SI-GA-G4_HWVT7BGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r5_SI-GA-G5_HMMKLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r5_SI-GA-G5_HMMMLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r5_SI-GA-G5_HW5MKBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r5_SI-GA-G5_HWVT7BGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r6_SI-GA-G6_HMMKLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r6_SI-GA-G6_HMMMLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r6_SI-GA-G6_HW5MKBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r6_SI-GA-G6_HWVT7BGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r1_CGTTACCG_HMMKLBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r1_CGTTACCG_HMMMLBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r1_CGTTACCG_HW5MKBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r2_ACTATTCA_HWVT7BGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r3_GACTGGAC_HMMKLBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r3_GACTGGAC_HMMMLBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r3_GACTGGAC_HW5MKBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r4_TTGCTTAG_HMMKLBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r4_TTGCTTAG_HMMMLBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r4_TTGCTTAG_HW5MKBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r5_AACGAATT_HMMKLBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r5_AACGAATT_HMMMLBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r5_AACGAATT_HW5MKBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r6_TCTATCGC_HMMKLBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r6_TCTATCGC_HMMMLBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r6_TCTATCGC_HW5MKBGX3bamtofastq,CRISPR Guide Capture

Writing /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi_lib.csv


In [8]:
%%writefile /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi_lib_v1.csv
fastqs,sample,library_type
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r1_SI-GA-G1_HMMKLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r1_SI-GA-G1_HMMMLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r1_SI-GA-G1_HW5MKBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r1_SI-GA-G1_HWVT7BGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r1_CGTTACCG_HMMKLBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r1_CGTTACCG_HMMMLBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r1_CGTTACCG_HW5MKBGX3bamtofastq,CRISPR Guide Capture

Writing /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi_lib_v1.csv


In [12]:
%%writefile /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi_lib_v2.csv
fastqs,sample,library_type
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r2_SI-GA-G2_HMMKLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r2_SI-GA-G2_HMMMLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r2_SI-GA-G2_HW5MKBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r2_SI-GA-G2_HWVT7BGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r2_ACTATTCA_HWVT7BGX3bamtofastq,CRISPR Guide Capture

Writing /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi_lib_v2.csv


In [1]:
%%writefile /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi_lib_v3.csv
fastqs,sample,library_type
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r3_SI-GA-G3_HMMKLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r3_SI-GA-G3_HMMMLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r3_SI-GA-G3_HW5MKBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r3_SI-GA-G3_HWVT7BGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r3_GACTGGAC_HMMKLBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r3_GACTGGAC_HMMMLBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r3_GACTGGAC_HW5MKBGX3bamtofastq,CRISPR Guide Capture

Writing /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi_lib_v3.csv


In [2]:
%%writefile /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi_lib_v4.csv
fastqs,sample,library_type
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r4_SI-GA-G4_HMMKLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r4_SI-GA-G4_HMMMLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r4_SI-GA-G4_HW5MKBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r4_SI-GA-G4_HWVT7BGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r4_TTGCTTAG_HMMKLBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r4_TTGCTTAG_HMMMLBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r4_TTGCTTAG_HW5MKBGX3bamtofastq,CRISPR Guide Capture

Writing /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi_lib_v4.csv


In [3]:
%%writefile /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi_lib_v5.csv
fastqs,sample,library_type
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r5_SI-GA-G5_HMMKLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r5_SI-GA-G5_HMMMLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r5_SI-GA-G5_HW5MKBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r5_SI-GA-G5_HWVT7BGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r5_AACGAATT_HMMKLBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r5_AACGAATT_HMMMLBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r5_AACGAATT_HW5MKBGX3bamtofastq,CRISPR Guide Capture

Writing /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi_lib_v5.csv


In [4]:
%%writefile /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi_lib_v6.csv
fastqs,sample,library_type
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r6_SI-GA-G6_HMMKLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r6_SI-GA-G6_HMMMLBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r6_SI-GA-G6_HW5MKBGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_cells_r6_SI-GA-G6_HWVT7BGX3bamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r6_TCTATCGC_HMMKLBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r6_TCTATCGC_HMMMLBGX3bamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi,K1000_CRISPRi_gRNA_r6_TCTATCGC_HW5MKBGX3bamtofastq,CRISPR Guide Capture

Writing /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi_lib_v6.csv


In [5]:
%%bash
mkdir -p /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/high_moi
cd /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/high_moi
sbatch \
    -p all \
    --exclude=dl-01 \
    -o /data/gersbachlab/Revathy/collabs/Susan/logs/high_moi_data_cellranger.v3.out \
    --mem=96G \
    --cpus-per-task=24 \
    <<'EOF'
#!/bin/bash
/data/gersbachlab/Ruhi/pipeline_perturbseq_like/cellranger-7.1.0/bin/cellranger count \
     --id="run1_r3" \
     --libraries=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi_lib_v3.csv \
     --transcriptome=/data/reddylab/Reference_Data/cellranger/refdata-gex-GRCh38-2020-A \
     --feature-ref=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi.feature_reference.csv \
     --chemistry=SC3Pv2 \
     --check-library-compatibility=false \
     --localcores=24 \
     --localmem=96 
EOF

Submitted batch job 30458975


In [6]:
%%bash
mkdir -p /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/high_moi
cd /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/high_moi
sbatch \
    -p all \
    --exclude=dl-01 \
    -o /data/gersbachlab/Revathy/collabs/Susan/logs/high_moi_data_cellranger.v4.out \
    --mem=96G \
    --cpus-per-task=24 \
    <<'EOF'
#!/bin/bash
/data/gersbachlab/Ruhi/pipeline_perturbseq_like/cellranger-7.1.0/bin/cellranger count \
     --id="run1_r4" \
     --libraries=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi_lib_v4.csv \
     --transcriptome=/data/reddylab/Reference_Data/cellranger/refdata-gex-GRCh38-2020-A \
     --feature-ref=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi.feature_reference.csv \
     --chemistry=SC3Pv2 \
     --check-library-compatibility=false \
     --localcores=24 \
     --localmem=96 
EOF

Submitted batch job 30458976


In [7]:
%%bash
mkdir -p /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/high_moi
cd /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/high_moi
sbatch \
    -p all \
    --exclude=dl-01 \
    -o /data/gersbachlab/Revathy/collabs/Susan/logs/high_moi_data_cellranger.v5.out \
    --mem=96G \
    --cpus-per-task=24 \
    <<'EOF'
#!/bin/bash
/data/gersbachlab/Ruhi/pipeline_perturbseq_like/cellranger-7.1.0/bin/cellranger count \
     --id="run1_r5" \
     --libraries=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi_lib_v5.csv \
     --transcriptome=/data/reddylab/Reference_Data/cellranger/refdata-gex-GRCh38-2020-A \
     --feature-ref=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi.feature_reference.csv \
     --chemistry=SC3Pv2 \
     --check-library-compatibility=false \
     --localcores=24 \
     --localmem=96 
EOF

Submitted batch job 30458977


In [9]:
%%bash
mkdir -p /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/high_moi
cd /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/high_moi
sbatch \
    -p all \
    --exclude=dl-01 \
    -o /data/gersbachlab/Revathy/collabs/Susan/logs/high_moi_data_cellranger.v6.out \
    --mem=96G \
    --cpus-per-task=24 \
    <<'EOF'
#!/bin/bash
/data/gersbachlab/Ruhi/pipeline_perturbseq_like/cellranger-7.1.0/bin/cellranger count \
     --id="run1_r6" \
     --libraries=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi_lib_v6.csv \
     --transcriptome=/data/reddylab/Reference_Data/cellranger/refdata-gex-GRCh38-2020-A \
     --feature-ref=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi.feature_reference.csv \
     --chemistry=SC3Pv2 \
     --check-library-compatibility=false \
     --localcores=24 \
     --localmem=96 
EOF

Submitted batch job 30458986


In [10]:
%%writefile /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi_lib.aggr.csv
sample_id,molecule_h5
run1_r1,/data/gersbachlab/Revathy/collabs/Susan/results/gasperini/high_moi/run1_r1/outs/molecule_info.h5
run1_r2,/data/gersbachlab/Revathy/collabs/Susan/results/gasperini/high_moi/run1_r2/outs/molecule_info.h5
run1_r3,/data/gersbachlab/Revathy/collabs/Susan/results/gasperini/high_moi/run1_r3/outs/molecule_info.h5
run1_r4,/data/gersbachlab/Revathy/collabs/Susan/results/gasperini/high_moi/run1_r4/outs/molecule_info.h5
run1_r5,/data/gersbachlab/Revathy/collabs/Susan/results/gasperini/high_moi/run1_r5/outs/molecule_info.h5
run1_r6,/data/gersbachlab/Revathy/collabs/Susan/results/gasperini/high_moi/run1_r6/outs/molecule_info.h5

Writing /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi_lib.aggr.csv


In [2]:
%%bash
cd /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/high_moi
sbatch \
    -p all \
    --exclude=dl-01 \
    -o /data/gersbachlab/Revathy/collabs/Susan/logs/high_moi_data_cellranger_aggr.out \
    --mem=48G \
    --cpus-per-task=24 \
    <<'EOF'
#!/bin/bash
/data/gersbachlab/Ruhi/pipeline_perturbseq_like/cellranger-7.1.0/bin/cellranger aggr \
     --localcores 24 \
     --localmem 48 \
     --normalize none \
     --id "aggr" \
     --csv /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/high_moi_lib.aggr.csv
     
EOF

Submitted batch job 30460651


## At scale data

Data folder: `/data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/bams/fastq`

In [3]:
%%bash
mkdir -p /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale
ln -s /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/bams/fastq /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale

In [30]:
!ls /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale/1A_8*

/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale/1A_8_gRNA_TAGAGGCG_MissingLibrary_1_H3FMKDSXXbamtofastq_S1_L001_I1_001.fastq.gz
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale/1A_8_gRNA_TAGAGGCG_MissingLibrary_1_H3FMKDSXXbamtofastq_S1_L001_R1_001.fastq.gz
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale/1A_8_gRNA_TAGAGGCG_MissingLibrary_1_H3FMKDSXXbamtofastq_S1_L001_R2_001.fastq.gz
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale/1A_8_gRNA_TAGAGGCG_MissingLibrary_1_H3FMKDSXXbamtofastq_S1_L002_I1_001.fastq.gz
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale/1A_8_gRNA_TAGAGGCG_MissingLibrary_1_H3FMKDSXXbamtofastq_S1_L002_R1_001.fastq.gz
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale/1A_8_gRNA_TAGAGGCG_MissingLibrary_1_H3FMKDSXXbamtofastq_S1_L002_R2_001.fastq.gz
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale/1A_8_gRNA_TAGAGGCG_MissingLibr

In [31]:
%%writefile /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/at_scale_lib_1A.csv
fastqs,sample,library_type
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1A_1_SI-GA-E2_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1A_2_SI-GA-E3_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1A_3_SI-GA-E4_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1A_4_SI-GA-E5_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1A_5_SI-GA-E6_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1A_6_SI-GA-E7_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1A_7_SI-GA-E8_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1A_8_SI-GA-E9_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1A_1_gRNA_CAGCTAGC_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1A_2_gRNA_CTAAGCCT_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1A_3_gRNA_CGTTACCG_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1A_4_gRNA_GACTGGAC_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1A_5_gRNA_GCAAGACC_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1A_6_gRNA_TCAATCTC_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1A_7_gRNA_ATACCTCG_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1A_8_gRNA_TAGAGGCG_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture

Overwriting /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/at_scale_lib_1A.csv


In [1]:
%%writefile /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/at_scale_lib_1B.csv
fastqs,sample,library_type
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1B_1_SI-GA-F2_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1B_2_SI-GA-F3_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1B_3_SI-GA-F4_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1B_4_SI-GA-F5_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1B_5_SI-GA-F6_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1B_6_SI-GA-F7_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1B_7_SI-GA-F8_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1B_8_SI-GA-F9_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1B_1_gRNA_TAGGTAAC_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1B_2_gRNA_TTCGAATA_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1B_3_gRNA_TGGACGAC_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1B_4_gRNA_GTAGGCTG_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1B_5_gRNA_GGTTATCG_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1B_6_gRNA_GCATCGTA_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1B_7_gRNA_AATACGAT_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,1B_8_gRNA_TTCCGTCG_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture

Writing /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/at_scale_lib_1B.csv


In [3]:
%%writefile /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/at_scale_lib_2A.csv
fastqs,sample,library_type
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2A_1_SI-GA-G2_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2A_2_SI-GA-G3_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2A_3_SI-GA-G4_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2A_4_SI-GA-G5_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2A_5_SI-GA-G6_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2A_6_SI-GA-G7_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2A_7_SI-GA-G8_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2A_8_SI-GA-G9_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2A_1_gRNA_AAGTAGCT_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2A_2_gRNA_TATTGCTG_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2A_3_gRNA_CCAGATAC_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2A_4_gRNA_AACGAATT_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2A_5_gRNA_CGCTTATC_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2A_6_gRNA_AAGTACGC_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2A_7_gRNA_GATCTTCG_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2A_8_gRNA_TCTTAGCC_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture

Writing /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/at_scale_lib_2A.csv


In [4]:
%%writefile /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/at_scale_lib_2B.csv
fastqs,sample,library_type
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2B_1_SI-GA-H2_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2B_2_SI-GA-H3_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2B_3_SI-GA-H4_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2B_4_SI-GA-H5_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2B_5_SI-GA-H6_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2B_6_SI-GA-H7_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2B_7_SI-GA-H8_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2B_8_SI-GA-H9_MissingLibrary_1_H3FMKDSXXbamtofastq,Gene Expression
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2B_1_gRNA_TTATTGAG_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2B_2_gRNA_TTGCGAGC_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2B_3_gRNA_GCTTGAAG_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2B_4_gRNA_AGTCCGCT_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2B_5_gRNA_ACGGCGTT_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2B_6_gRNA_GGCTTACT_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2B_7_gRNA_GCGCGTTC_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture
/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale,2B_8_gRNA_GAGCGCGA_MissingLibrary_1_H3FMKDSXXbamtofastq,CRISPR Guide Capture

Writing /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/at_scale_lib_2B.csv


#### Feature ref for at_scale data

In [2]:
%%bash
#Download and processing gasperini guide sequences
wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE120nnn/GSE120861/suppl/GSE120861_grna_groups.at_scale.txt.gz; gunzip --force -d GSE120861_grna_groups.at_scale.txt.gz; awk '{ print $2 "\t" $1}' GSE120861_grna_groups.at_scale.txt > raw_gasperini_at_scale_guide_sequences.tsv
head -n 10 raw_gasperini_at_scale_guide_sequences.tsv  

AAACCGCTCCCGAGCACGGG	SH3BGRL3_TSS
AAATAGTGGGAAGATTCGTG	MTRNR2L8_TSS
AACACACCACGGAGGAGTGG	FAM83A_TSS
AACAGCCCGGCCGGCCAAGG	ZNF593_TSS
AACGAGAGACTGCTTGCTGG	ATPIF1_TSS
AACGGCTCGGAAGCCTAGGG	TIPRL_TSS
AAGAAGAAACCGAGAGACGG	MYL4_TSS
AAGACGGCTCGCAAGGTAGG	CNBP_TSS
AAGAGGAAGGTGAGCCTCTG	RPS3_TSS
AAGATGTCGAAGCGAGGTGG	RPL23_TSS


--2023-12-11 12:19:00--  https://ftp.ncbi.nlm.nih.gov/geo/series/GSE120nnn/GSE120861/suppl/GSE120861_grna_groups.at_scale.txt.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.10, 130.14.250.7, 2607:f220:41e:250::7, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 127092 (124K) [application/x-gzip]
Saving to: ‘GSE120861_grna_groups.at_scale.txt.gz’

     0K .......... .......... .......... .......... .......... 40% 1.62M 0s
    50K .......... .......... .......... .......... .......... 80% 3.15M 0s
   100K .......... .......... ....                            100% 88.1M=0.05s

2023-12-11 12:19:01 (2.65 MB/s) - ‘GSE120861_grna_groups.at_scale.txt.gz’ saved [127092/127092]



In [3]:
import pandas as pd

pd_guide_target = pd.read_csv('raw_gasperini_at_scale_guide_sequences.tsv',sep='\t', header=None, names=['seq', 'target'])
add_guide_additional_id = {}
final_guide_names = []
for g in pd_guide_target['target']:
    if g not in add_guide_additional_id:
        final_guide_names.append(g+'|1')
        add_guide_additional_id[g]=True
    else:
        final_guide_names.append(g+'|2')
pd_guide_target['target'] = final_guide_names

In [7]:
pd_guide_target.to_csv('gasperini_at_scale_guide_sequences.tsv', sep='\t', index=None, header=None)

In [8]:
%%bash
mkdir -p /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/at_scale
cd /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/at_scale
sbatch \
    -p all \
    --exclude=dl-01 \
    -o /data/gersbachlab/Revathy/collabs/Susan/logs/at_scale_data_cellranger.v1.out \
    --mem=96G \
    --cpus-per-task=24 \
    <<'EOF'
#!/bin/bash
/data/gersbachlab/Ruhi/pipeline_perturbseq_like/cellranger-7.1.0/bin/cellranger count \
     --id="run1A" \
     --libraries=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/at_scale_lib_1A.csv \
     --transcriptome=/data/reddylab/Reference_Data/cellranger/refdata-gex-GRCh38-2020-A \
     --feature-ref=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/feature_reference.at_scale.csv \
     --chemistry=SC3Pv2 \
     --check-library-compatibility=false \
     --localcores=24 \
     --localmem=96 
EOF

Submitted batch job 30469792


In [9]:
%%bash
mkdir -p /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/at_scale
cd /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/at_scale
sbatch \
    -p all \
    --exclude=dl-01 \
    -o /data/gersbachlab/Revathy/collabs/Susan/logs/at_scale_data_cellranger.v2.out \
    --mem=96G \
    --cpus-per-task=24 \
    <<'EOF'
#!/bin/bash
/data/gersbachlab/Ruhi/pipeline_perturbseq_like/cellranger-7.1.0/bin/cellranger count \
     --id="run1B" \
     --libraries=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/at_scale_lib_1B.csv \
     --transcriptome=/data/reddylab/Reference_Data/cellranger/refdata-gex-GRCh38-2020-A \
     --feature-ref=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/feature_reference.at_scale.csv \
     --chemistry=SC3Pv2 \
     --check-library-compatibility=false \
     --localcores=24 \
     --localmem=96 
EOF

Submitted batch job 30469793


In [10]:
%%bash
mkdir -p /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/at_scale
cd /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/at_scale
sbatch \
    -p all \
    --exclude=dl-01 \
    -o /data/gersbachlab/Revathy/collabs/Susan/logs/at_scale_data_cellranger.v3.out \
    --mem=96G \
    --cpus-per-task=24 \
    <<'EOF'
#!/bin/bash
/data/gersbachlab/Ruhi/pipeline_perturbseq_like/cellranger-7.1.0/bin/cellranger count \
     --id="run2A" \
     --libraries=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/at_scale_lib_2A.csv \
     --transcriptome=/data/reddylab/Reference_Data/cellranger/refdata-gex-GRCh38-2020-A \
     --feature-ref=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/feature_reference.at_scale.csv \
     --chemistry=SC3Pv2 \
     --check-library-compatibility=false \
     --localcores=24 \
     --localmem=96 
EOF

Submitted batch job 30469794


In [11]:
%%bash
mkdir -p /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/at_scale
cd /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/at_scale
sbatch \
    -p all \
    --exclude=dl-01 \
    -o /data/gersbachlab/Revathy/collabs/Susan/logs/at_scale_data_cellranger.v4.out \
    --mem=96G \
    --cpus-per-task=24 \
    <<'EOF'
#!/bin/bash
/data/gersbachlab/Ruhi/pipeline_perturbseq_like/cellranger-7.1.0/bin/cellranger count \
     --id="run2B" \
     --libraries=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/at_scale_lib_2B.csv \
     --transcriptome=/data/reddylab/Reference_Data/cellranger/refdata-gex-GRCh38-2020-A \
     --feature-ref=/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/feature_reference.at_scale.csv \
     --chemistry=SC3Pv2 \
     --check-library-compatibility=false \
     --localcores=24 \
     --localmem=96 
EOF

Submitted batch job 30469795


In [1]:
%%writefile /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/at_scale_lib.aggr.csv
sample_id,molecule_h5
run1A,/data/gersbachlab/Revathy/collabs/Susan/results/gasperini/at_scale/run1A/outs/molecule_info.h5
run1B,/data/gersbachlab/Revathy/collabs/Susan/results/gasperini/at_scale/run1B/outs/molecule_info.h5
run2A,/data/gersbachlab/Revathy/collabs/Susan/results/gasperini/at_scale/run2A/outs/molecule_info.h5
run2B,/data/gersbachlab/Revathy/collabs/Susan/results/gasperini/at_scale/run2B/outs/molecule_info.h5

Writing /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/at_scale_lib.aggr.csv


In [1]:
%%bash
cd /data/gersbachlab/Revathy/collabs/Susan/results/gasperini/at_scale
sbatch \
    -p all \
    --exclude=dl-01 \
    -o /data/gersbachlab/Revathy/collabs/Susan/logs/at_scale_data_cellranger_aggr.out \
    --mem=48G \
    --cpus-per-task=24 \
    <<'EOF'
#!/bin/bash
/data/gersbachlab/Ruhi/pipeline_perturbseq_like/cellranger-7.1.0/bin/cellranger aggr \
     --localcores 24 \
     --localmem 48 \
     --normalize none \
     --id "aggr" \
     --csv /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/metadata/at_scale_lib.aggr.csv
     
EOF

Submitted batch job 30496362


-----
-----
-----


In [1]:
%%writefile /data/reddylab/Alex/collab/20181203_Tyler/data/metadata/gasperini_et_all_cell_2019.at_scale_grna_geo.txt
GSE_ID	GSM_ID	Library_ID
GSE120861	GSM3417256	K562.scrnaseq.sgrna.at_scale


Writing /data/reddylab/Alex/collab/20181203_Tyler/data/metadata/gasperini_et_all_cell_2019.at_scale_grna_geo.txt


In [ ]:
%%%bash
export TMPDIR="/data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/tmp"
# in hardac_xfer
while read -r a b c; do
    python /data/reddylab/Alex/reddylab_utils/scripts/download_from_geo.py     \
        -gse ${a}     \
        -gsms ${b}   \
        -o /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/ 
done < <(tail -n+2 /data/reddylab/Alex/collab/20181203_Tyler/data/metadata/gasperini_et_all_cell_2019.at_scale_grna_geo.txt | cut -f 1,2,3)


The above command didn't work, it kep crushing. I ended up using this instead:

In [ ]:
%%bash
    cd /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/tmp
for ii in $(seq 7967526 7967556); 
do 
    echo $ii; 
    /data/reddylab/software/sratoolkit/sratoolkit.2.11.1-centos_linux64/bin/fasterq-dump SRR${ii} \
    && gzip SRR${ii}.fastq; 
done;

In [1]:
%%bash
mkdir -p /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/logs
mkdir -p /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/counts

Actually, trying the data in the ENCODE portal 

In [ ]:
%%bash
cd 
curl -u PJCLYEZS:eqzl3qepb6pzzkub -SLO https://www.encodeproject.org/files/ENCFF142PRC/@@download/ENCFF142PRC.fastq.hz

In [9]:
%%bash
cd /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/counts
#7967526-7967557%10
sbatch \
    --array=26 \
    -p all \
    -o /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/logs/sgrna%a_10x_processing.out \
    --mem=64G \
    --cpus-per-task=24 \
    <<'EOF'
#!/bin/bash
/data/gersbachlab/Software/cellranger-3.1.0/cellranger count \
    --id=SRR79675${SLURM_ARRAY_TASK_ID} \
    --fastq=/data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/Supp_GSM3417256_At_Scale_Screen_gRNA_enrichment_scRNA-seq/ \
    --transcriptome=/data/gersbachlab/Software/refdata-cellranger-hg19-3.0.0/ \
    --sample=SRR79675${SLURM_ARRAY_TASK_ID} \
    --expect-cells=7000 \
    --localcores=24 \
    --localmem=64 \
    --chemistry=auto
EOF

Submitted batch job 26208719


```
Though it has been a while, according to the methods and my memory, the gRNA protospacer should be in the same fastqs. These libraries were separately indexed and then sequenced as spike-ins alongside the whole transcriptome scRNA-seq libraries.

To access them, we used a script written by Andrew Hill (CC'd). He can confirm, but I copied and pasted below from the paper where this analysis method was first published.

For your understanding, barcode = protospacer.

- Molly

Assigning Cell Genotypes

Barcode enrichment libraries were separately indexed and sequenced as spike-ins alongside the whole transcriptome scRNA-seq libraries. Final UMI and cell barcode assignments were made for each read by processing these samples with cellranger 1.3.1 as was done for the whole transcriptome libraries.

A whitelist of guide or target barcode sequences was constructed using all guides or target barcodes in the library. For each read in the position-sorted BAM file output by cellranger 1.3.1, the final cell barcode and UMI are extracted. If either of these fields is not populated, indicating a problem with the sequence, the read is ignored. Using the cDNA read, we attempt to find a perfect match for the sequence preceding the guide or barcode (GTGGAAAGGACGAAACACCG for CROP-seq and CGCCTCCCGCG for pLGB-scKO). If a perfect match is not found, we attempt to locate the sequence using a striped Smith-Watterman alignment. If a match or alignment is found, the guide or barcode sequence is extracted. If the extracted sequence does not perfectly match a whitelist sequence, we search for a matching whitelist sequence within an edit distance of half the minimum edit distance between any pair of guides or barcodes in the library (rounded down). If no match is found, the molecule is ultimately discarded. Matches to the whitelist are tracked for each cell.

We also remove likely chimeric sequences using the approach outlined in Dixit24. Briefly, within each cell, we calculate the number of times a given UMI is observed with each observed guide assignment. We then divide these counts by the total instances of the respective UMI across all observed guide assignments within that cell. For UMI-guide assignment combinations where this fraction is less than 20%, we do not count the UMI towards the final observed guide assignment counts. While this has some impact on the raw data, we find the benefits to be modest.

To make a set of final assignments, we take all whitelist sequences with over 10 reads and that account for over 7.5% of the whitelist reads assigned to a given cell, where multiple sequences can be assigned to each cell. This set of assignments is merged with the filtered gene expression matrices output by cellranger such that only assignments to the filtered cells appear in the final dataset.
```

In fact, ENCODE has the samples in 4 pools. Download them from there and process as Tylers

In [ ]:
%%bash
xfer
for ii in $(cut -f1 molly_sc_screen.metadata.txt | tail -n+2); 
do 
    if [[ ! -e ${ii}.fastq.gz ]]; 
    then 
        nohup bash -c "curl -L  -u PJCLYEZS:eqzl3qepb6pzzkub -o ${ii}.fastq.gz https://www.encodeproject.org/files/${ii}/@@download/${ii}.fastq.gz > ${ii}.out &"; 
    fi; 
done

In [1]:
#%%bash
cd /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode
rename ENCFF722ETK K562.CRISPRi.pool2.R1 ENCFF722ETK*
rename ENCFF277LKQ K562.CRISPRi.pool2.R2 ENCFF277LKQ*
rename ENCFF706ISZ K562.CRISPRi.pool3.R1 ENCFF706ISZ*
rename ENCFF485YMB K562.CRISPRi.pool3.R2 ENCFF485YMB*
rename ENCFF142PRC K562.CRISPRi.pool4.R1 ENCFF142PRC*
rename ENCFF878FJZ K562.CRISPRi.pool4.R2 ENCFF878FJZ*
rename ENCFF837WTS K562.CRISPRi.pool1.R1 ENCFF837WTS*
rename ENCFF302WYY K562.CRISPRi.pool1.R2 ENCFF302WYY*

# I remember (a touch too late) that cellranger expects FASTQ file names in a certain way 
rename K562.CRISPRi.pool2.R1 K562.CRISPRi.pool2_S2_L002_R1_001 K562.CRISPRi.pool2.R1*
rename K562.CRISPRi.pool2.R2 K562.CRISPRi.pool2_S2_L002_R2_001 K562.CRISPRi.pool2.R2*
rename K562.CRISPRi.pool3.R1 K562.CRISPRi.pool3_S3_L003_R1_001 K562.CRISPRi.pool3.R1*
rename K562.CRISPRi.pool3.R2 K562.CRISPRi.pool3_S3_L003_R2_001 K562.CRISPRi.pool3.R2*
rename K562.CRISPRi.pool4.R1 K562.CRISPRi.pool4_S4_L004_R1_001 K562.CRISPRi.pool4.R1*
rename K562.CRISPRi.pool4.R2 K562.CRISPRi.pool4_S4_L004_R2_001 K562.CRISPRi.pool4.R2*
rename K562.CRISPRi.pool1.R1 K562.CRISPRi.pool1_S1_L001_R1_001 K562.CRISPRi.pool1.R1*
rename K562.CRISPRi.pool1.R2 K562.CRISPRi.pool1_S1_L001_R2_001 K562.CRISPRi.pool1.R2*

SyntaxError: invalid token (<ipython-input-1-7e7d00def355>, line 2)

In [12]:
%%bash
cd /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/
mkdir -p counts
cd counts
sbatch \
    --array=2-4 \
    -o /data/reddylab/Alex/collab/20181203_Tyler/logs/gasperini.pool%a_10x_processing.out \
    --mem=64G \
    --cpus-per-task=16 \
    <<'EOF'
#!/bin/bash
/data/gersbachlab/Software/cellranger-3.1.0/cellranger count \
    --id=pool${SLURM_ARRAY_TASK_ID} \
    --fastq=/data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/fastq/ \
    --transcriptome=/data/gersbachlab/Software/refdata-cellranger-hg19-3.0.0/ \
    --sample=pool${SLURM_ARRAY_TASK_ID} \
    --localcores=16 \
    --localmem=64 \
    --chemistry=SC3Pv2
EOF

Submitted batch job 26894415


----

Download guides

In [1]:
%%bash
cd /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/
mkdir fasta
cd fasta
curl -u PJCLYEZS:eqzl3qepb6pzzkub -SLO https://www.encodeproject.org/files/ENCFF292NTB/@@download/ENCFF292NTB.fasta.gz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1622  100  1622    0     0   2403      0 --:--:-- --:--:-- --:--:--  2402
100  178k  100  178k    0     0   132k      0  0:00:01  0:00:01 --:--:-- 35.3M


Index them with Bowtie2

In [3]:
%%bash
module load bowtie2
cd /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/fasta
bowtie2-build \
    ENCFF292NTB.fasta.gz \
    gasperini_et_al.guides_bt2

Settings:
  Output files: "gasperini_et_al.guides_bt2.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  ENCFF292NTB.fasta.gz
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 65754
Using parameters --bmax 49316 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 49316 --dcv 1024
Constructing suffix-array element

Building a SMALL index


In [ ]:
samtools bam2fq \
        -0 /data/gersbachlab/mia/sc_ceres_seq_NTPool/count/sgrnas/${SAMPLE}/outs/possorted_genome_bam.fastq \
        -f 4 \
        /data/gersbachlab/mia/sc_ceres_seq_NTPool/count/sgrnas/${SAMPLE}/outs/possorted_genome_bam.bam \
&& gzip /data/gersbachlab/mia/sc_ceres_seq_NTPool/count/sgrnas/${SAMPLE}/outs/possorted_genome_bam.fastq
samtools bam2fq \
    -0 /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/possorted_genome_bam.fastq \
    -f 4 \
    /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/possorted_genome_bam.bam \
&& gzip /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/possorted_genome_bam.fastq


In [6]:
%%bash
module load bowtie2
module load samtools
sbatch -p all \
    --mem 32G \
    --cpus-per-task 8 \
    --array 2-4 \
    --output /data/reddylab/Alex/collab/20181203_Tyler/logs/gasperini_et_al.bowtie2.extract_umi_counts_from_grna_bam.pool_%a.out \
    <<'EOF'
#!/bin/bash
if [[ ! -e /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/possorted_genome_bam.fastq ]];
then
    samtools bam2fq \
        -0 /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/possorted_genome_bam.fastq \
        -f 4 \
        /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/possorted_genome_bam.bam \
    && gzip /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/possorted_genome_bam.fastq
fi
bowtie2 \
    --trim5 23 \
    --trim3 48 \
    --no-unal \
    --end-to-end \
    -D 15 -R 2 -N 1 -L 18 -i S,1,0 \
    --score-min G,0,0 \
    --ignore-quals \
    -x /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/fasta/gasperini_et_al.guides_bt2 \
    -U /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/possorted_genome_bam.fastq.gz \
    --threads 8 \
| samtools view -b \
> /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/possorted_genome_bam.bt2.bam
EOF


Submitted batch job 26897490


In [2]:
%%bash
module load samtools
sbatch \
    --array=1-4 \
    -p all \
    -o /data/reddylab/Alex/collab/20181203_Tyler/logs/gasperini_et_al.sc_crispr_i.create_UB_fastq.%a.out \
<<'EOF'
#!/bin/bash
samtools view -f4 \
/data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/possorted_genome_bam.bam \
| /bin/grep UB: | /bin/grep CB: \
| awk -vOFS='\n' '{print "@"$1, gensub(/.*UB:Z:([CAGT]+)[[:space:]].*/, "\\1", "g", $0), "+", "EEEEEEEEEE"}' \
> /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/possorted_genome_bam.UB.fastq ;
EOF


Submitted batch job 26971613


In [6]:
%%bash
module load samtools
sbatch \
    --array=1-4 \
    -p all \
    -o /data/reddylab/Alex/collab/20181203_Tyler/logs/gasperini_et_al.sc_crispr_i.create_CB_fastq.%a.out \
<<'EOF'
#!/bin/bash
samtools view -f4 \
/data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/possorted_genome_bam.bam \
| /bin/grep UB: | /bin/grep CB: \
| awk -vOFS='\n' '{print "@"$1, gensub(/.*CB:Z:(.+)\-1.*/, "\\1", "g", $0), "+", "EEEEEEEEEEEEEEEE"}' \
> /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/possorted_genome_bam.CB.fastq ;
EOF


Submitted batch job 26970306


The script to assign tags to read in the BAMs is running out of memory... We need to split the BAMs, assign the tags, and merge the BAMs back into a single BAM...

In [ ]:
COMBOS=($(echo {1,2,3,4}@{chr1,chr2,chr3,chr4,chr5,chr6,chr7,chr8,chr9,chr10,chr11,chr12,chr13,chr14,chr15,chr16,chr17,chr18,chr19,chr20,chr21,chr22,chrX})
COMBO=${COMBOS[${SLURM_ARRAY_TASK_ID}]}
POOL=$(echo ${COMBO} | cut -d@ -f1)
CHROM=$(echo ${COMBO} | cut -d@ -f2)


In [5]:
%%bash
module load samtools
sbatch \
    --array=1 \
    --cpus-per-task 8 \
    --mem=8G \
    -pall \
    -o /data/reddylab/Alex/collab/20181203_Tyler/logs/gasperini_et_al.sc_crispr_i.sort_and_index_pool_bam.%a.out \
<<'EOF'
#!/bin/bash
POOL=${SLURM_ARRAY_TASK_ID}
DATA_DIR="/data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${POOL}/outs"

samtools sort \
    -@ 8 \
    -T /data/reddylab/Alex/tmp/gasperini_et_al.sc_crispr_i.sort_bt2 \
    ${DATA_DIR}/possorted_genome_bam.bt2.bam \
> ${DATA_DIR}/possorted_genome_bam.bt2.sorted.bam \
&& samtools index ${DATA_DIR}/possorted_genome_bam.bt2.sorted.bam

EOF

Submitted batch job 27102719


In [3]:
%%bash
module load samtools
COMBOS=($(echo {1,2,3,4}@{chr1,chr2,chr3,chr4,chr5,chr6,chr7,chr8,chr9,chr10,chr11,chr12,chr13,chr14,chr15,chr16,chr17,chr18,chr19,chr20,chr21,chr22,chrX}))
sbatch \
    --array=0 \
    --mem=8G \
    -pall \
    -o /data/reddylab/Alex/collab/20181203_Tyler/logs/gasperini_et_al.sc_crispr_i.split_pool_bam_by_chrom.%a.out \
<<'EOF'
#!/bin/bash
COMBOS=($(echo {1,2,3,4}@{chr1,chr2,chr3,chr4,chr5,chr6,chr7,chr8,chr9,chr10,chr11,chr12,chr13,chr14,chr15,chr16,chr17,chr18,chr19,chr20,chr21,chr22,chrX}))
COMBO=${COMBOS[${SLURM_ARRAY_TASK_ID}]}
POOL=$(echo ${COMBO} | cut -d@ -f1)
CHROM=$(echo ${COMBO} | cut -d@ -f2)
DATA_DIR="/data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${POOL}/outs"
mkdir -p ${DATA_DIR}/by_chorm/${CHROM} 

samtools view \
    -b \
    ${DATA_DIR}/possorted_genome_bam.bt2.sorted.bam \
    ${CHROM} \
> ${DATA_DIR}/by_chorm/possorted_genome_bam.bt2.${CHROM}.bam
EOF


Submitted batch job 27102716


In [ ]:
java -Xmx32g \
    -jar /data/reddylab/software/fgbio/0.8.1/fgbio.jar AnnotateBamWithUmis \
    -i /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/possorted_genome_bam.bt2.bam \
    -o /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/split_data/possorted_genome_bam.bt2.CB.0000.bam \
    -f /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/split_data/possorted_genome_bam.CB.0000.fastq \
    -t CB


Ok, so not the most efficient way to do this, but path forward. The problem is the `AnnotateBamWithUmis` commmand load in memory the whole set of FASTQ reads containing the UMI/barcodes. Somehow it must also duplicate some other data structures becasue it choked with 240G of RAM. The solution seems to be to reduce the size of the FASTQ file containing UMI/barcodes.
- [X] Split UB and CB fastq files in 50 chunks
- [ ] Annotate the reads contained in each FASTQ file iteratively, so that the last file has the annotations of all previous files. Clean up any temporary file in the process.
- [ ] Clean up all secondary files

In [16]:
%%bash
module load java
sbatch \
    --array=2-4 \
    --mem=8G \
    -p all \
    -o /data/reddylab/Alex/collab/20181203_Tyler/logs/gasperini_et_al.sc_crispr_i.split_fastq_files.v2.%a.out \
<<'EOF'
#!/bin/bash

mkdir -p /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/split_data_v2/UB
mkdir -p /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/split_data_v2/CB
cd /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/
split -d -l 562800000 --additional-suffix=.fastq possorted_genome_bam.UB.fastq split_data_v2/UB/possorted_genome_bam.UB.
split -d -l 562800000 --additional-suffix=.fastq possorted_genome_bam.CB.fastq split_data_v2/CB/possorted_genome_bam.CB.
EOF

Submitted batch job 27121909


In [5]:
%%bash
for POOL in 2 3 4;
do
    ln -s /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${POOL}/outs/possorted_genome_bam.bt2.bam \
        /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${POOL}/outs/split_data/UB/possorted_genome_bam.bt2.UB.00.bam
done

In [17]:
%%bash
module load java
sbatch \
    --exclude=dl-01 \
    --depend=afterok:27121909 \
    --array=2-4 \
    --mem=96G \
    -pall \
    -o /data/reddylab/Alex/collab/20181203_Tyler/logs/gasperini_et_al.sc_crispr_i.merge_bam_with_tags.split_version.v02.%a.out \
<<'EOF'
#!/bin/bash

ln -s /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/possorted_genome_bam.bt2.bam \
    /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/split_data_v2/UB/possorted_genome_bam.bt2.UB.00.bam
cd /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/split_data_v2/UB
for ii in *[0-9].fastq; 
do
    BATCH=$(echo ${ii} | cut -d. -f3)
    TMPBATCH=$((10#${BATCH}))
    NEXTBATCH=$(printf "%02d" $((${TMPBATCH} + 1)))
    time java -Xmx64g \
        -jar /data/reddylab/software/fgbio/0.8.1/fgbio.jar AnnotateBamWithUmis \
        -i possorted_genome_bam.bt2.UB.${BATCH}.bam \
        -o possorted_genome_bam.bt2.UB.${NEXTBATCH}.bam \
        -f possorted_genome_bam.UB.${BATCH}.fastq \
        -t UB
    echo "possorted_genome_bam.bt2.UB.${NEXTBATCH}.bam finished"
    rm -f possorted_genome_bam.bt2.UB.${BATCH}.bam
done

cd /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/split_data_v2/CB
ln -s ../UB/possorted_genome_bam.bt2.UB.${NEXTBATCH}.bam ./possorted_genome_bam.bt2.CB.00.bam
for ii in *[0-9].fastq; 
do
    BATCH=$(echo ${ii} | cut -d. -f3)
    TMPBATCH=$((10#${BATCH}))
    NEXTBATCH=$(printf "%02d" $((${TMPBATCH} + 1)))
    time java -Xmx64g \
        -jar /data/reddylab/software/fgbio/0.8.1/fgbio.jar AnnotateBamWithUmis \
        -i possorted_genome_bam.bt2.CB.${BATCH}.bam \
        -o possorted_genome_bam.bt2.CB.${NEXTBATCH}.bam \
        -f possorted_genome_bam.CB.${BATCH}.fastq \
        -t CB
    > /dev/null 2>&1
    echo "possorted_genome_bam.bt2.CB.${NEXTBATCH}.bam finished"
    rm -f possorted_genome_bam.bt2.CB.${BATCH}.bam
done

EOF


Submitted batch job 27121913


^-- This worked, with simply 5 batches of FASTQ reads

Now, use custom script to re-create guide MM files

In [4]:
%%bash
ls /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/poo11/outs/split_data_v2/CB/*bam

/data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool1/outs/split_data_v2/CB/possorted_genome_bam.bt2.CB.05.bam
/data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool2/outs/split_data_v2/CB/possorted_genome_bam.bt2.CB.06.bam
/data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool3/outs/split_data_v2/CB/possorted_genome_bam.bt2.CB.06.bam
/data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool4/outs/split_data_v2/CB/possorted_genome_bam.bt2.CB.06.bam


In [6]:
len("GGATGACTCAACAGAATGGG")

20

In [9]:
%%bash
cut -d_ -f2 /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/fasta/protospacer_ids.txt \
    > /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/fasta/protospacer_ids.simple.txt

In [1]:
%%bash
source /data/reddylab/software/miniconda3/bin/activate alex_py3
module load bzip2
sbatch -pall \
    --exclude=dl-01 \
    --mem 64G \
    --array 3 \
    --output /data/reddylab/Alex/collab/20181203_Tyler/logs/gasperini_et_al.scRNAseq.extract_umi_counts_from_grna_bam.sam_with_tags.pool_%a.out \
    <<'EOF'
#!/bin/bash
# _pool${SLURM_ARRAY_TASK_ID}
#     --sam /data/reddylab/Alex/collab/20181203_Tyler/sc_crispr_i/counts/pool${SLURM_ARRAY_TASK_ID}/outs/possorted_genome_bam.bam \
mkdir -p  /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/new
python /data/reddylab/Alex/collab/20181203_Tyler/scripts/scRNAseq.extract_umi_counts_from_grna_bam.py \
    --sam /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/split_data_v2/CB/possorted_genome_bam.bt2.CB.06.bam \
    --sam-has-all-tags \
    --protospacers /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/fasta/protospacer_ids.simple.txt \
    --outdir /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/new \
    --cell-barcodes /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool${SLURM_ARRAY_TASK_ID}/outs/filtered_feature_bc_matrix/barcodes.tsv.gz \
    --cell-barcode-tag CB \
    --umi-tag UB \
    --outname pool${SLURM_ARRAY_TASK_ID} \
    --sam-flag 4 \
    --cell-pool ${SLURM_ARRAY_TASK_ID}
EOF


Submitted batch job 27247257


In [ ]:
# %load /data/reddylab/Alex/collab/20181203_Tyler/scripts/scRNAseq.extract_umi_counts_from_grna_bam.py
#!/usr/bin/env python3 
import argparse
import fileinput
import sys
from collections import defaultdict as dd
import re
import numpy as np
from skbio.alignment import StripedSmithWaterman as ssm
import pandas as pd
import pysam

def revcomp(s):
    comp = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    return ''.join([comp[c] for c in s[::-1]])

def sw(protospacers, query_seq):
    s = ssm(query_seq)
    for target_seq in protospacers:
        res = s(target_seq)
        if '20M' in res.cigar or '19M' in res.cigar:
            return target_seq
    else:
        return None

def match_within_distance(protospacers, query_seq, max_distance = 1):
    alphabet = ['A', 'T', 'C', 'G']
    distance = 0
    if query_seq in protospacers:
        return query_seq, distance
    while distance < max_distance:
        distance += 1
        for nucl_ix in range(len(query_seq)):
            for base in [a for a in alphabet if a != query_seq[nucl_ix]]:
                mutant = query_seq[:nucl_ix] + base + query_seq[nucl_ix+1:]
                if mutant in protospacers:
                    return mutant, distance
    return None, -1
    

def main():
    #"/data/reddylab/Alex/tmp/foo.test.sam"
    parser = argparse.ArgumentParser(formatter_class=argparse.RawTextHelpFormatter, 
                                     description="""""")

    optional = parser._action_groups.pop()
    required = parser.add_argument_group('required arguments')

    ##################################################
    # required args:
    required.add_argument("--protospacers", type=str, required=True,
                          help="File containing gRNA protospacers to search for.")
    required.add_argument("--sam", type=str, required=True,
                          help="SAM/BAM file")
    required.add_argument("--outdir", type=str, required=True,
                          help="Output directory where files will be created (or overwritten if existing)")
    required.add_argument("--outname", type=str, required=True,
                          help="String used as basename for generated files")
    # required.add_argument("strategy", type=str, choices=['exact-all', 'partial-last'], 
    #                       help="required, FASTQ R1 file")
    ##################################################
    # optional args:
    optional.add_argument("--sam-has-all-tags", action='store_true', required=False,
                          help="Use if SAM/BAM has already the UMI (default: UB) and cell barcodes (default: CB) tags")
    optional.add_argument("--sam-flag", type=int, required=False,
                          help="Use SAM flag to select matching alignments")
    optional.add_argument("--protospacer-start-pos", type=str, required=False, default=23,
                          help="Position where protospacer starts. (default: 23)")
    optional.add_argument("--protospacer-length", type=str, required=False, default=20,
                          help="Protospacer length. (default: 20)")
    optional.add_argument("--cell-barcodes", type=str, required=False,
                          help="If provided, use this file with one barcode per line instead of the observed ones")
    optional.add_argument("--cell-barcode-tag", type=str, required=False, default='CB',
                          help="By default, use the tag CB as cellular identifier. Other tags can be used (like CR)")
    optional.add_argument("--umi-tag", type=str, required=False, default='UB',
                          help="By default, use the tag UB as molecular identifier. Other tags can be used (like UR)")
    optional.add_argument("--cell-pool", type=int, required=False,
                          help="If provided, add this int to the barcodes in this pool")

    parser._action_groups.append(optional)
    # /data/reddylab/Alex/collab/20181203_Tyler/data/external/scRNAseq/possorted_genome_bam.sam
    # args = parser.parse_args("--protospacers /data/reddylab/Alex/collab/20181203_Tyler/data/external/scRNAseq/scLib-gRNAs.protospacers.txt --sam /data/reddylab/Alex/collab/20181203_Tyler/data/external/scRNAseq/possorted_genome_bam.sam".split())
    args = parser.parse_args()

    # parse command line
    protospacer_start_pos = args.protospacer_start_pos
    protospacers = pd.read_csv(args.protospacers, header=None).values.T.tolist()[0]
    cell_barcodes = []
    
    if args.cell_barcodes:
        cell_barcodes = pd.read_csv(args.cell_barcodes, header=None).values.T.tolist()[0]
    else:
        if args.sam_has_all_tags:
            for a in pysam.AlignmentFile(args.sam, "rb"):
                # sequence = a.query_sequence
                try:
                    cell_barcode = dict(a.tags)[args.cell_barcode_tag]
                except KeyError:
                    continue
                cell_barcodes.append(cell_barcode)
            cell_barcodes = list(set(cell_barcodes))
        else:
            for a in pysam.AlignmentFile(args.sam, "rb"):
                if args.sam_flag and not (args.sam_flag & a.flag == args.sam_flag):
                    continue
                try:
                    cell_barcode = dict(a.tags)[args.cell_barcode_tag]
                except KeyError:
                    continue
                cell_barcodes.append(cell_barcode)
            cell_barcodes = list(set(cell_barcodes))
    
    print ("Barcodes loaded...")
    mm = dd(list)
    nbarcodes_notfound = 0
    if args.sam_has_all_tags:
        for a in pysam.AlignmentFile(args.sam, "rb"):
            try:
                cell_barcode = dict(a.tags)[args.cell_barcode_tag]
                umi = dict(a.tags)[args.umi_tag]
            except KeyError:
                continue
            observed_protospacer = a.query_sequence
            protospacers_ix = protospacers.index(observed_protospacer)
            try:
                if '-' not in cell_barcode:
                    cell_barcode = "%s-1" % cell_barcode
                cell_barcodes_ix = cell_barcodes.index(cell_barcode)
            except ValueError:
                nbarcodes_notfound += 1
                continue # Barcode not present in provided list
            key = "%d_%d" % (protospacers_ix, cell_barcodes_ix)
            mm[key].append(umi)
        print ("Number of cell barcodes present in gRNA BAM not found in barcode whitelist: %d" % nbarcodes_notfound)
    else:
        for a in pysam.AlignmentFile(args.sam, "rb"):
            if args.sam_flag and not (args.sam_flag & a.flag == args.sam_flag):
                continue
            try:
                cell_barcode = dict(a.tags)[args.cell_barcode_tag]
                umi = dict(a.tags)[args.umi_tag]
            except KeyError:
                continue
            sequence = a.query_sequence
            if 16 & a.flag == 16:
                sequence  = revcomp(sequence)

            observed_protospacer = sequence[protospacer_start_pos:protospacer_start_pos+args.protospacer_length]
            if observed_protospacer not in protospacers:
                # If protospacer not found in normal position, try +/- 1bp:
                for offset in [-1, 1, -2, 2]:
                    observed_protospacer = sequence[protospacer_start_pos+offset:protospacer_start_pos+offset+args.protospacer_length]
                    if observed_protospacer in protospacers:
                        break
                else: # protospacer not found, try Smith-Waterman
    #                 continue
    #                 observed_protospacer = sw(protospacers, sequence[protospacer_start_pos:protospacer_start_pos+args.protospacer_length])
                    for offset in [0, -1, 1]:
                        observed_protospacer, distance = \
                            match_within_distance(protospacers, 
                                                  sequence[protospacer_start_pos+offset:protospacer_start_pos+offset+args.protospacer_length], 
                                                  max_distance=1)
                        if observed_protospacer:
                            break
            if not observed_protospacer:
                continue
            protospacers_ix = protospacers.index(observed_protospacer)
            try:
                cell_barcodes_ix = cell_barcodes.index(cell_barcode)
            except ValueError:
                continue # Barcode not present in provided list
            key = "%d_%d" % (protospacers_ix, cell_barcodes_ix)
            mm[key].append(umi)

    print ("Observed protospacers stored...")
    mm_avg = dd(list)
    mm_median = dd(list)
    
    for k in mm.keys():
        mm[k] = len(set(mm[k]))
        mm_avg[k].append(np.average(np.unique(np.array(mm[k]), return_counts=True)[1]))
        mm_median[k].append(np.median(np.unique(np.array(mm[k]), return_counts=True)[1]))

    with open('%s/%s.matrix.tsv' % (args.outdir, args.outname), 'w') as ofile:
        ofile.write("%%MatrixMarket matrix coordinate integer general\n")
        ofile.write("%metadata_json: {\"format_version\": 2, \"software_version\": \"3.1.0\"}\n")
        ofile.write("%d\t%d\t%d\n" % (len(protospacers), len(cell_barcodes), len(mm.keys())))
        for k, v in mm.items():
            ps, cell_barcode = [int(kk)+1 for kk in k.split("_")]
            ofile.write("%d\t%d" % (ps, cell_barcode))
            ofile.write("\t%d\n" % v)

    pool_suffix = '-%d' % args.cell_pool if  args.cell_pool else ''
    pd.DataFrame([c.split('-')[0] + pool_suffix for c in cell_barcodes])\
        .to_csv('%s/%s.barcodes.tsv' % (args.outdir, args.outname),
                index=False, header=False)

    # Save average and median UMI counts per cell (since we only count multi-read UMIs as 1)
    pd.DataFrame.from_dict(mm_avg).T.to_csv('%s/%s.counts_per_umi.avg.txt' % (args.outdir, args.outname), header=None, sep='\t')
    pd.DataFrame.from_dict(mm_median).T.to_csv('%s/%s.counts_per_umi.median.txt' % (args.outdir, args.outname), header=None, sep='\t')
        
if __name__ == '__main__':
    main()


Create 3 files per sample with the exact filenames expected by the `Read10X` function 

In [1]:
%%bash
cd  /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/
for POOL in 1 2 3 4;
do
    gzip -c pool${POOL}/outs/new/pool${POOL}.matrix.tsv > pool${POOL}/outs/new/matrix.mtx.gz 
    gzip -c pool${POOL}/outs/new/pool${POOL}.barcodes.tsv > pool${POOL}/outs/new/barcodes.tsv.gz 
    gzip -c /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/fasta/protospacer_ids.simple.txt \
        > pool${POOL}/outs/new/features.tsv.gz 
done


In [2]:
!wc -l /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/fasta/protospacer_ids.txt 

13189 /data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/fasta/protospacer_ids.txt


In R (alex_py3)

In [ ]:
suppressPackageStartupMessages(library(argparse))
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(GenomicRanges))
suppressPackageStartupMessages(library(stringr))
suppressPackageStartupMessages(library(SparseData))
suppressPackageStartupMessages(library(purrr))

filepath <- c('/data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool1/outs/new/', 
              '/data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool2/outs/new/',
              '/data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool3/outs/new/',
              '/data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/pool4/outs/new/')
gRNAs.counts<- purrr::reduce(purrr::map(filepath, Read10X, gene.column=1), combine)
writeMM(gRNAs.counts, file='/data/reddylab/Alex/collab/20181203_Tyler/data/external/gasperini/encode/counts/gRNAs.all.MM.txt.gz')

In [ ]:
function(M.list) {
  A = M.list[[1]]

  for (i in 2:length(M.list)){ #i indexes of matrices
    # finding what's missing
    misA = colnames(M.list[[i]])[!colnames(M.list[[i]]) %in% colnames(A)]
    misB = colnames(A)[!colnames(A) %in% colnames(M.list[[i]])]

    misAl = as.vector(numeric(length(misA)), "list")
    names(misAl) = misA
    misBl = as.vector(numeric(length(misB)), "list")
    names(misBl) = misB

    ## adding missing columns to initial matrices
    An = Reduce(cbind, c(A, misAl))
    if (length(misA) > 0)
       {
       lenA <- ncol(An)-length(misA)+1
       colnames(An)[lenA:ncol(An)] = names(misAl)
       }

    Bn = Reduce(cbind, c(M.list[[i]], misBl))
    if(length(misB) > 0)
       {
       lenB <- ncol(Bn)-length(misB)+1
       colnames(Bn)[lenB:ncol(Bn)] = names(misBl)
       }

    Bn <- Bn[,colnames(An)]

    # final bind
    A = rbind(An, Bn, use.names = T)
    print(c(length(M.list), i))
  }
  A
}
gRNAs.counts<- purrr::map(filepath, Read10X, gene.column=1, strip.suffix = T)
merged <- merge.sparse(gRNAs.counts)